# Deep Learning hw1

## Adrian Hsu

In [42]:
# Deep Learning hw1

from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile

from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as cPickle

Q. python語法問題，什麼是 "_" ? 

A. As a general purpose "throwaway" variable name to indicate that part of a function result is being deliberately ignored, as in code like:

In [43]:
text = "1:2"
label, has_label, _ = text.partition(':')
print(label)

1


In [44]:
url = 'http://yaroslavvb.com/upload/notMNIST/'

def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or (not os.path.exists(filename)):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz


In [45]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  print("root is: " + root)
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall()
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

root is: notMNIST_large
notMNIST_large already present - Skipping extraction of notMNIST_large.tar.gz.
['notMNIST_large/A', 'notMNIST_large/B', 'notMNIST_large/C', 'notMNIST_large/D', 'notMNIST_large/E', 'notMNIST_large/F', 'notMNIST_large/G', 'notMNIST_large/H', 'notMNIST_large/I', 'notMNIST_large/J']
root is: notMNIST_small
notMNIST_small already present - Skipping extraction of notMNIST_small.tar.gz.
['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']


### disp_samples在做的事情
for notMNIST_large/A 到 notMNIST_large/F  
先選出 notMNIST_large/A  
listdir可以把/A裡面的所有照片名稱列出，存到image_files  
從裡面挑出其中2張image  
用os.path.join合併出從root算起的path,   
e.g. notMNIST_large/A/TmV3QmFza2VydmlsbGVFRi1Sb21hbi5vdGY=.png  
有了path後，用Image包住，然後display  

In [46]:
import random
import hashlib
%matplotlib inline

In [47]:
def disp_samples(data_folders, sample_size):
  for folder in data_folders:
    print(folder) # folder "notMNIST_large/A"
    image_files = os.listdir(folder) # 列出folder "notMNIST_large/A"裡面所有照片的path(A LOT!)
    image_sample = random.sample(image_files, sample_size) #image_sample 為選定的image名稱string
    print(image_sample)
    for image in image_sample:
      image_file = os.path.join(folder, image) # path/image名稱
      print("image_file: " + image_file)
      i = Image(image_file) # filename=image_file, 省略filename
      display(i)
    

In [48]:
disp_samples(train_folders, 1)

notMNIST_large/A
['WXVrb24gVGVjaCBFeHBhbmRlZC50dGY=.png']
image_file: notMNIST_large/A/WXVrb24gVGVjaCBFeHBhbmRlZC50dGY=.png


notMNIST_large/B
['SE9aRU5PWlpMRS50dGY=.png']
image_file: notMNIST_large/B/SE9aRU5PWlpMRS50dGY=.png


notMNIST_large/C
['WEJvYnNFeHRyYVBpY3R1cmVzLnR0Zg==.png']
image_file: notMNIST_large/C/WEJvYnNFeHRyYVBpY3R1cmVzLnR0Zg==.png


notMNIST_large/D
['SGFsdmVyc29uIEhlYXZ5IEl0YWxpYy50dGY=.png']
image_file: notMNIST_large/D/SGFsdmVyc29uIEhlYXZ5IEl0YWxpYy50dGY=.png


notMNIST_large/E
['U2FpbnRMb3Vpc0JvbGQudHRm.png']
image_file: notMNIST_large/E/U2FpbnRMb3Vpc0JvbGQudHRm.png


notMNIST_large/F
['Um9ja2V0IFR5cGUgU2hhZG93LnR0Zg==.png']
image_file: notMNIST_large/F/Um9ja2V0IFR5cGUgU2hhZG93LnR0Zg==.png


notMNIST_large/G
['V2FsYmF1bVN0YW5kYXJkRXhwZXJ0QlEtUmVndWxhci5vdGY=.png']
image_file: notMNIST_large/G/V2FsYmF1bVN0YW5kYXJkRXhwZXJ0QlEtUmVndWxhci5vdGY=.png


notMNIST_large/H
['S1IgRWFzdGVyIE5vIDIudHRm.png']
image_file: notMNIST_large/H/S1IgRWFzdGVyIE5vIDIudHRm.png


notMNIST_large/I
['Qm9kb25pQmVydGhvbGRCUS1NZWRpdW0ub3Rm.png']
image_file: notMNIST_large/I/Qm9kb25pQmVydGhvbGRCUS1NZWRpdW0ub3Rm.png


notMNIST_large/J
['UG90YXNzaXVtIFNjYW5kYWwudHRm.png']
image_file: notMNIST_large/J/UG90YXNzaXVtIFNjYW5kYWwudHRm.png


In [49]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  image_index = 0
  print(folder)
  for image in os.listdir(folder):
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[image_index, :, :] = image_data
      image_index += 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  num_images = image_index
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

Pickling notMNIST_large/A.pickle.
notMNIST_large/A


ImportError: Could not import the Python Imaging Library (PIL) required to load image files.  Please refer to http://pypi.python.org/pypi/PIL/ for installation instructions.